In [1]:
%pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [ ]:
# make all the bigrams for this entire corpus
# there's a method in nltk that does this for us!
review_bigrams = nltk.bigrams(movie_reviews.words())

In [8]:
# let's extract all the bigrams where the context token is "people"
from collections import Counter
# finall all the target tokens that appear after "people"
target_tokens = []
for bigram in review_bigrams:
    if bigram[0].lower() == 'people':
        target_token = bigram[1].lower()
        target_tokens.append(target_token)

people_bigram_lm = Counter(target_tokens)
# Print the top 10 most common bigrams that start with "people"
print(people_bigram_lm.most_common(10))

[('chasing', 1)]
